In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
matchups = pd.read_csv("data\\matchups\\model_matchups2.csv", index_col=0).dropna()

In [3]:
y = matchups['outcome']
x_train = matchups.drop(columns=['outcome','inning','game_code','batter_game_code','pitcher_game_code','batter_id','pitcher_id','b_pos'])

In [4]:
from sklearn import preprocessing
from sklearn.decomposition import PCA

# Change outcome labels to one-hot vectors
le = preprocessing.LabelEncoder()
le.fit(y)
labels_enc = le.transform(y)
labels = tf.keras.utils.to_categorical(labels_enc)

# Apply standard scaling to features
ss = preprocessing.StandardScaler()
x_standardized = ss.fit_transform(x_train)

# Reduce dimensionality of input data
# pca = PCA(n_components=50)
# principal_components = pca.fit_transform(x_standardized)
# x_standardized = pd.DataFrame(data = principal_components).to_numpy()

In [5]:
p = .05
inputs = tf.keras.layers.Input((73,), name='numeric_inputs')

x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(1500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(250, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(11, activation='softmax', name='output')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 numeric_inputs (InputLayer)  [(None, 73)]             0         
                                                                 
 dropout (Dropout)           (None, 73)                0         
                                                                 
 dense (Dense)               (None, 1500)              111000    
                                                                 
 batch_normalization (BatchN  (None, 1500)             6000      
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 1500)              0         
                                                                 
 dense_1 (Dense)             (None, 500)               750500    
                                                             

In [6]:
def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(x_standardized.shape[0], batch_size)
        yield ({'numeric_inputs': x_standardized[batch_idx]},
               {'output': labels[batch_idx]})

batch_size = 128

model.fit(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=10,
    max_queue_size=10  
)


Epoch 1/10
78/78 [==============================] - 2s 10ms/step - loss: 1.7457 - accuracy: 0.4938
Epoch 2/10
78/78 [==============================] - 1s 10ms/step - loss: 1.2850 - accuracy: 0.5697
Epoch 3/10
78/78 [==============================] - 1s 10ms/step - loss: 1.2010 - accuracy: 0.5544
Epoch 4/10
78/78 [==============================] - 1s 10ms/step - loss: 1.1208 - accuracy: 0.5608
Epoch 5/10
78/78 [==============================] - 1s 10ms/step - loss: 1.1143 - accuracy: 0.5617
Epoch 6/10
78/78 [==============================] - 1s 10ms/step - loss: 1.1057 - accuracy: 0.5573
Epoch 7/10
78/78 [==============================] - 1s 10ms/step - loss: 1.0783 - accuracy: 0.5694
Epoch 8/10
78/78 [==============================] - 1s 10ms/step - loss: 1.0647 - accuracy: 0.5691
Epoch 9/10
78/78 [==============================] - 1s 10ms/step - loss: 1.0578 - accuracy: 0.5712
Epoch 10/10
78/78 [==============================] - 1s 10ms/step - loss: 1.0357 - accuracy: 0.5792
